In [ ]:
%matplotlib inline
import sys
from datetime import date, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('/Users/jorge/Projects/finance')

from portfolio import Portfolio
from components.tickers import Tickers

In [ ]:
p = Portfolio()
t = Tickers()

days = p.performance.keys()
values = []
deposits = []
profit = []
max_value = 0
yesterday = date.today() - timedelta(days=1)

for day, data in p.performance.items():
    values.append(data['marketValue'])
    deposits.append(data['totalDeposits'])
    profit.append(float(data['profitOrLoss']))
    max_value = max(max_value, float(data['marketValue']))

    print(f"{day}: {data['marketValue']:10,.2f}")

In [ ]:
sns.set()
sns.set_style('whitegrid')
fig, axes = plt.subplots(6, 1, figsize=(16, 50))

plt.subplot(611)
plt.ylabel('Amounts')
ylim = max_value + 5000 - (max_value % 5000)
plt.ylim(0, ylim)
plt.title('Market Value')
values_df = pd.DataFrame(values, index=days, columns=['Values'])
deposits_df = pd.DataFrame(deposits, index=days, columns=['Deposits'])
plt.plot(values_df, '#40C040', deposits_df, '#6060B0', linewidth=1.0)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
yticks = ax.get_yticks()
ax.set_yticklabels(['{:10,.0f}'.format(y) for y in yticks])
ax.legend(['Market Value', 'Deposits'])

plt.subplot(612)
plt.title('Profit or Loss')
profit_df = pd.DataFrame(profit, index=days, columns=['Profit'])
zeros = pd.DataFrame(1.0, index=days, columns=['Zero'])
ax = plt.gca()
plt.plot(profit_df, 'CornflowerBlue', linewidth=1.0)
ax.autoscale(axis='x', tight=True)
ax.fill_between(profit_df.index, profit_df['Profit'], color='CornflowerBlue', alpha=0.8)
yticks = ax.get_yticks()
ax.set_yticklabels(['{:10,.0f}'.format(y) for y in yticks])
ax.legend(['Profit or Loss'])

plt.subplot(613)
plt.title('Money-, Time-Weighted, and Total Returns')
plt.ylabel('Returns')
mwrr = pd.DataFrame([x['mwrr'] for x in p.performance.values()], index=days, columns=['mwrr'])
ttwr = pd.DataFrame([x['ttwr'] for x in p.performance.values()], index=days, columns=['ttwr'])
returns = pd.DataFrame([x['totalReturns'] for x in p.performance.values()], index=days, columns=['returns'])
plt.plot(mwrr, '#40C040', ttwr, '#C09030', returns, '#80A0C0', linewidth=1.0)
plt.ylim(-0.04, 0.20)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
ax.set_yticks([k / 100.0 for k in range(-4, 20, 2)])
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(['Money-Weighted Real Returns', 'True Time-Weighted Returns', 'Total Returns'])
    
plt.subplot(614)
plt.title('Annualized Money- and Time-Weighted Returns')
plt.ylabel('Returns')
plt.ylim(-0.05, 0.3)
mwrr = pd.DataFrame([x['mwrrAnnualized'] for x in p.performance.values()], index=days, columns=['mwrrAnnualized'])
ttwr = pd.DataFrame([x['ttwrAnnualized'] for x in p.performance.values()], index=days, columns=['ttwrAnnualized'])
plt.plot(mwrr, '#40C040', ttwr, '#C09030', linewidth=1.0)
plt.ylim(-0.04, 0.20)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
ax.set_yticks([k / 100.0 for k in range(-4, 20, 2)])
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(['Annualized Money-Weighted Real Returns', 'Annualized True Time-Weighted Returns'])
    
plt.subplot(615)
plt.title('Ticker Returns')
plt.plot(t.returns, linewidth=1.0)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(t.returns.columns)

plt.subplot(616)
plt.title('Correlations')

mask = np.zeros_like(t.correlations, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
ax = plt.gca()

# Generate a custom diverging colormap
cmap = sns.diverging_palette(240, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(t.correlations, mask=mask, cmap=cmap, vmin=-1, vmax=1, square=True,
            linewidths=.5, cbar_kws={"shrink": .8}, annot=True, ax=ax)

plt.show()

In [ ]:
p.performance[yesterday]

In [ ]:
allocations = pd.DataFrame(
    [[x, round(float(y['percentPortfolio'] * 100), 2)] for x, y in p.performance[date.today() - timedelta(days=1)]['assets'].items()],
    columns=['Asset', 'Allocation'])
allocations = allocations.set_index('Asset').sort_index()

cash_allocation = pd.DataFrame(
    [('Cash', round(float(p.performance[date.today() - timedelta(days=1)]['percentCash'] * 100), 2))],
    columns=['Asset', 'Allocation'])
cash_allocation = cash_allocation.set_index('Asset')

allocations = allocations.append(cash_allocation)

allocations